In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [ ]:
def load_csvs(folder):
    data_path = f"C:\\path\\to\\data\\{folder}"
    df = pd.read_csv(f"{data_path}\\super_res_test.csv", index_col=0, header=[0, 1])
    textures = pd.read_csv(f"{data_path}\\super_res_test_texture.csv", index_col=0)

    def _change_expt_name(expt):
        expt = folder.replace("_nores", "") + '_' +  expt
        return expt

    df.columns = pd.MultiIndex.from_tuples([(c[0], _change_expt_name(c[1])) for c in df.columns])
    textures.columns = [_change_expt_name(c) for c in textures.columns]

    return df, textures

In [ ]:
quality_deep, texture_deep = load_csvs("deep")
quality_shallow, texture_shallow = load_csvs("shallow")
quality_multi, texture_multi = load_csvs("multi")
quality_df = pd.concat([quality_deep, quality_shallow, quality_multi], axis=1)
texture_df = pd.concat([texture_deep, texture_shallow, texture_multi], axis=1)
texture_df.drop(labels=["shallow_LQ", "shallow_HQ", "multi_LQ", "multi_HQ"], axis=1, inplace=True)
texture_df.rename({"deep_LQ": "LQ", "deep_HQ": "HQ"}, axis=1, inplace=True)

quality_deep, texture_deep = load_csvs("deep_nores")
quality_shallow, texture_shallow = load_csvs("shallow_nores")
quality_multi, texture_multi = load_csvs("multi_nores")
quality_nores_df = pd.concat([quality_deep, quality_shallow, quality_multi], axis=1)
texture_nores_df = pd.concat([texture_deep, texture_shallow, texture_multi], axis=1)
texture_nores_df.drop(labels=["shallow_LQ", "shallow_HQ", "multi_LQ", "multi_HQ"], axis=1, inplace=True)
texture_nores_df.rename({"deep_LQ": "LQ", "deep_HQ": "HQ"}, axis=1, inplace=True)

# Remove outliers
exclude = ["T066A0LQ023", "T066A0LQ024", "T066A0LQ032", "T066A0LQ034", "T066A0LQ047", "T066A0LQ049", "T066A0LQ061", "T066A0LQ063", "T066A0LQ084", "T066A0LQ086", "T066A0LQ099", "T066A0LQ100"]
quality_df = quality_df.loc[~quality_df.index.isin(exclude)]
texture_df = texture_df.loc[~texture_df.index.isin(exclude)]
quality_nores_df = quality_nores_df.loc[~quality_nores_df.index.isin(exclude)]
texture_nores_df = texture_nores_df.loc[~texture_nores_df.index.isin(exclude)]

In [ ]:
# expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]
# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq128", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq64", "shallow_jo_vq1024", "shallow_jo_vq4"]
# expts = ["multi_sr_vq0", "multi_sr_vq128-256-512-1024-1024", "multi_jo_vq32-128-512-128-64", "multi_jo_vq128-256-512-1024-256"]

names = ["Base", "VQ-SR", "JointVQ-SR", "JointVQ-CE"]
output = ""
df = quality_df

for name, expt in zip(names, expts):
    output += f"{name}"

    for metric, sig in {"MSE": 0, "pSNR": 2, "SSIM": 4}.items():
        median = np.round(df.loc[:, pd.IndexSlice[metric, expt]].median(), sig)
        output += f" & {median} "
        quantiles = np.round(df.loc[:, pd.IndexSlice[metric, expt]].quantile([0.05, 0.95]), sig)
        output += f"{list(quantiles)}"

    output += "\\\ \n"

print(output)

In [ ]:
# expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]
expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
# expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq128", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq64", "shallow_jo_vq1024", "shallow_jo_vq4"]
# expts = ["multi_sr_vq0", "multi_sr_vq128-256-512-1024-1024", "multi_jo_vq32-128-512-128-64", "multi_jo_vq128-256-512-1024-256"]

names = ["Base", "VQ-SR", "JointVQ-SR", "JointVQ-CE"]
output = ""
df = texture_df

for name, expt in zip(names, expts):
    output += f"{name}"

    median = np.round(df.loc[:, expt].median(), 2)
    output += f" & {median} "
    quantiles = np.round(df.loc[:, expt].quantile([0.05, 0.95]), 2)
    output += f"{list(quantiles)}"

    output += "\\\ \n"

print(output)

print(texture_nores_df.loc[:, ["LQ", "HQ"]].quantile([0.05, 0.5, 0.95]))

In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(3, 1, 1)
plt.boxplot(quality_df["MSE"])
plt.xticks(ticks=np.arange(1, quality_df["MSE"].shape[1] + 1), labels=quality_df["MSE"].columns, rotation="vertical")
plt.subplot(3, 1, 2)
plt.boxplot(quality_df["pSNR"])
plt.xticks(ticks=np.arange(1, quality_df["pSNR"].shape[1] + 1), labels=quality_df["pSNR"].columns, rotation="vertical")
plt.subplot(3, 1, 3)
plt.boxplot(quality_df["SSIM"])
plt.xticks(ticks=np.arange(1, quality_df["SSIM"].shape[1] + 1), labels=quality_df["SSIM"].columns, rotation="vertical")
plt.tight_layout()
plt.show()

In [ ]:
print("Baselines")
print(f"Alpha = {0.05 / 3}")
expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]

print(f"MSE: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

print(f"MSE: {stats.kruskal(*quality_nores_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR: {stats.kruskal(*quality_nores_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM: {stats.kruskal(*quality_nores_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

print("SSIM base non-resid")
print(f"SSIM deep vs shallow: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'deep_sr_vq0']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'shallow_sr_vq0']])}")
print(f"SSIM deep vs multi: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'deep_sr_vq0']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq0']])}")
print(f"SSIM shallow vs multi: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'shallow_sr_vq0']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq0']])}")

In [ ]:
print(f"Alpha = {0.05 / 6}")
# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

print(f"MSE: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM: {stats.kruskal(*quality_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq128", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq64", "shallow_jo_vq1024", "shallow_jo_vq4"]
expts = ["multi_sr_vq0", "multi_sr_vq128-256-512-1024-1024", "multi_jo_vq32-128-512-128-64", "multi_jo_vq128-256-512-1024-256"]

print(f"MSE: {stats.kruskal(*quality_nores_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR: {stats.kruskal(*quality_nores_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM: {stats.kruskal(*quality_nores_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

print("SSIM multi non-resid")
print(f"SSIM base vs vq-sr: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq0']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq128-256-512-1024-1024']])}")
print(f"SSIM base vs jvq-sr: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq0']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_jo_vq32-128-512-128-64']])}")
print(f"SSIM base vs jvq-ce: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq0']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_jo_vq128-256-512-1024-256']])}")
print(f"SSIM vq-sr vs jvq-sr: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq128-256-512-1024-1024']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_jo_vq32-128-512-128-64']])}")
print(f"SSIM vq-sr vs jvq-ce: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_sr_vq128-256-512-1024-1024']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_jo_vq128-256-512-1024-256']])}")
print(f"SSIM jvq-sr vs jvq-ce: {stats.mannwhitneyu(quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_jo_vq32-128-512-128-64']], quality_nores_df.loc[:, pd.IndexSlice['SSIM', 'multi_jo_vq128-256-512-1024-256']])}")

In [ ]:
print("Baselines")
print(f"Alpha = {0.05 / 3}")
expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]

print("Residual")
print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")

print("Non-residual")
print(f"Texture: {stats.kruskal(*texture_nores_df.loc[:, expts].values.T.tolist())}")

print(f"Texture base vs shallow: {stats.mannwhitneyu(texture_nores_df.loc[:, 'deep_sr_vq0'], texture_nores_df.loc[:, 'shallow_sr_vq0'])}")
print(f"Texture base vs multi: {stats.mannwhitneyu(texture_nores_df.loc[:, 'deep_sr_vq0'], texture_nores_df.loc[:, 'multi_sr_vq0'])}")
print(f"Texture shallow vs multi: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_sr_vq0'], texture_nores_df.loc[:, 'multi_sr_vq0'])}")

In [ ]:
print(f"Alpha = {0.05 / 18}")

# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq64", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq128", "shallow_jo_vq8", "shallow_jo_vq256"]
expts = ["multi_sr_vq0", "multi_sr_vq128-128-256-512-512", "multi_jo_vq128-64-256-32-512", "multi_jo_vq128-128-512-512-1024"]

print("Residual")
print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")

print("Multi residual")
print(f"Texture base vs vq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq0'], texture_df.loc[:, 'multi_sr_vq128-128-256-512-512'])}")
print(f"Texture base vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq0'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture base vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq0'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture vq-sr vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture vq-sr vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_sr_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture jvq-sr vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")

# expts = ["deep_sr_vq0", "deep_sr_vq1", "deep_jo_vq128", "deep_jo_vq512"]
# expts = ["shallow_sr_vq0", "shallow_sr_vq64", "shallow_jo_vq1024", "shallow_jo_vq4"]
expts = ["multi_sr_vq0", "multi_sr_vq128-256-512-1024-1024", "multi_jo_vq32-128-512-128-64", "multi_jo_vq128-256-512-1024-256"]

print("Non-residual")
print(f"Texture: {stats.kruskal(*texture_nores_df.loc[:, expts].values.T.tolist())}")

print("Shallow non-resid")
print(f"Texture base vs vq-sr: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_sr_vq0'], texture_nores_df.loc[:, 'shallow_sr_vq64'])}")
print(f"Texture base vs jvq-sr: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_sr_vq0'], texture_nores_df.loc[:, 'shallow_jo_vq1024'])}")
print(f"Texture base vs jvq-ce: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_sr_vq0'], texture_nores_df.loc[:, 'shallow_jo_vq4'])}")
print(f"Texture vq-sr vs jvq-sr: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_sr_vq64'], texture_nores_df.loc[:, 'shallow_jo_vq1024'])}")
print(f"Texture vq-sr vs jvq-ce: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_sr_vq64'], texture_nores_df.loc[:, 'shallow_jo_vq4'])}")
print(f"Texture jvq-sr vs jvq-ce: {stats.mannwhitneyu(texture_nores_df.loc[:, 'shallow_jo_vq1024'], texture_nores_df.loc[:, 'shallow_jo_vq4'])}")

print("Multi non-resid")
print(f"Texture base vs vq-sr: {stats.mannwhitneyu(texture_nores_df.loc[:, 'multi_sr_vq0'], texture_nores_df.loc[:, 'multi_sr_vq128-256-512-1024-1024'])}")
print(f"Texture base vs jvq-sr: {stats.mannwhitneyu(texture_nores_df.loc[:, 'multi_sr_vq0'], texture_nores_df.loc[:, 'multi_jo_vq32-128-512-128-64'])}")
print(f"Texture base vs jvq-ce: {stats.mannwhitneyu(texture_nores_df.loc[:, 'multi_sr_vq0'], texture_nores_df.loc[:, 'multi_jo_vq128-256-512-1024-256'])}")
print(f"Texture vq-sr vs jvq-sr: {stats.mannwhitneyu(texture_nores_df.loc[:, 'multi_sr_vq128-256-512-1024-1024'], texture_nores_df.loc[:, 'multi_jo_vq32-128-512-128-64'])}")
print(f"Texture vq-sr vs jvq-ce: {stats.mannwhitneyu(texture_nores_df.loc[:, 'multi_sr_vq128-256-512-1024-1024'], texture_nores_df.loc[:, 'multi_jo_vq128-256-512-1024-256'])}")
print(f"Texture jvq-sr vs jvq-ce: {stats.mannwhitneyu(texture_nores_df.loc[:, 'multi_jo_vq32-128-512-128-64'], texture_nores_df.loc[:, 'multi_jo_vq128-256-512-1024-256'])}")

In [ ]:
print("Baselines")
print(f"Alpha = {0.05 / 3}")
expts = ["deep_sr_vq0", "shallow_sr_vq0", "multi_sr_vq0"]

print()

for expt in expts:
    print(f"MSE {expt} resid vs non-resid: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['MSE', expt]], quality_nores_df.loc[:, pd.IndexSlice['MSE', expt]])}")
    print(f"pSNR {expt} resid vs non-resid: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['pSNR', expt]], quality_nores_df.loc[:, pd.IndexSlice['pSNR', expt]])}")
    print(f"SSIM {expt} resid vs non-resid: {stats.mannwhitneyu(quality_df.loc[:, pd.IndexSlice['SSIM', expt]], quality_nores_df.loc[:, pd.IndexSlice['SSIM', expt]])}")

print()

for expt in expts:
    print(f"Texture {expt} resid vs non-resid: {stats.mannwhitneyu(texture_df.loc[:, expt], texture_nores_df.loc[:, expt])}")